In [2]:
# Load LSTM network and generate text
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.utils.data_utils import get_file

# load ascii text and covert to lowercase
path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
raw_text = open(path).read().lower()

# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)
print("")

# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))

# normalize
X = X / float(n_vocab)

# one hot encode the output variable
y = np_utils.to_categorical(dataY)

# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# define the checkpoint
filepath="weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

# fit the model
model.fit(X, y, nb_epoch=1, batch_size=128, callbacks=callbacks_list)

Total Characters:  600893
Total Vocab:  57
Total Patterns:  600793
Instructions for updating:
This op will be removed after the deprecation date. Please switch to tf.unstack.
Instructions for updating:
This op will be removed after the deprecation date. Please switch to tf.stack.
Epoch 1/1
600793/600793 [==============================] - 484s - loss: 2.8416   


In [3]:
# load the network weights
filename = "weights.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
print("")

# generate characters
for i in range(500):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
"  not stand fast, boscovich has taught us to abjure the belief in the
last thing that "stood fast" of "
e soet the hesee tf the seree tf the seree tf the seree tf the seree tf the seree tf the seree tf the seree tf the seree tf the seree tf the seree tf the seree tf the seree tf the seree tf the seree tf the seree tf the seree tf the seree tf the seree tf the seree tf the seree tf the seree tf the seree tf the seree tf the seree tf the seree tf the seree tf the seree tf the seree tf the seree tf the seree tf the seree tf the seree tf the seree tf the seree tf the seree tf the seree tf the seree tf
Done.
